# Working with APIs



Lesson Goals

    Understand what is API and what it does.
    Learn how to make simple calls to an API and retrieve JSON data.
    Learn how to handle nested JSON API results.

Introduction

Thus far in the program, we have learned how to obtain data from files and from relational databases. However, sometimes the data we need is not readily available via one of these two data sources. In some cases, the data we need may be contained within an application. Application owners will often create APIs (or Application Programming Interface) so that their applications can talk to other applications. An API is a set of programmatic instructions for accessing software applications, and the data that comes from APIs typically contains some sort of structure (such as JSON). This structure makes working with API data preferable to crawling websites and scraping content off of web pages.

In this lesson, we are going to learn how to make API calls to an application, retrieve data in JSON format, learn about API authentication, and use Python libraries to obtain data from APIs.
Simple API Example with Requests

There are a few libraries that can be used for working with APIs in Python, but the Requests library is one of the most intuitive. It has a get method that allows you to send an HTTP request to an application and receive a response. Let's take a look at a basic API call using the requests library. 

In [1]:
import json
import requests

response = requests.get('https://jsonplaceholder.typicode.com/todos')
results = response.json()
results[0]

{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}

In this example, we used the get method to send a request to the JSONPlaceholder API, and we received back a response in the form of JSON structured data. If we wanted to analyze this data, we could easily use Pandas to convert the results into a data frame to which we can then apply various analytical methods. 

In [2]:
import pandas as pd

data = pd.DataFrame(results)
data.head()

,userId,id,title,completed
0,1,1,delectus aut autem,False
1,1,2,quis ut nam facilis et officia qui,False
2,1,3,fugiat veniam minus,False
3,1,4,et porro tempora,True
4,1,5,laboriosam mollitia et enim quasi adipisci qui...,False


# More Complex Requests API Example

In the previous section, the data we received from the API was not very complex. It was all at a single level and fit neatly into a data frame. However, sometimes API responses contain data that is nested, and we must find a way to flatten the JSON data so that it fits nicely into a data frame. Let's make an API call to the Github public API, create a Pandas data frame from the results, and examine the structure of the data.



In [3]:
response = requests.get('https://api.github.com/events')

data = pd.DataFrame(response.json())
data.head()

,id,type,actor,repo,payload,public,created_at,org
0,10785261173,PushEvent,"{'id': 18611758, 'login': 'victorjourne', 'dis...","{'id': 218634132, 'name': 'victorjourne/test',...","{'push_id': 4233036375, 'size': 1, 'distinct_s...",True,2019-11-04T23:06:42Z,NaN
1,10785261172,PushEvent,"{'id': 29196449, 'login': 'nickdurbin', 'displ...","{'id': 218709067, 'name': 'nickdurbin/auth-fri...","{'push_id': 4233036378, 'size': 1, 'distinct_s...",True,2019-11-04T23:06:42Z,NaN
2,10785261161,PushEvent,"{'id': 19600854, 'login': 'hwx438865', 'displa...","{'id': 202865472, 'name': 'hwx438865/carengine...","{'push_id': 4233036367, 'size': 1, 'distinct_s...",True,2019-11-04T23:06:42Z,NaN
3,10785261145,PullRequestEvent,"{'id': 41453073, 'login': 'concow', 'display_l...","{'id': 181036257, 'name': 'learn-co-students/c...","{'action': 'opened', 'number': 102, 'pull_requ...",True,2019-11-04T23:06:42Z,"{'id': 8825476, 'login': 'learn-co-students', ..."
4,10785261158,CreateEvent,"{'id': 57375669, 'login': 'Aletse-Meza', 'disp...","{'id': 219613610, 'name': 'Aletse-Meza/AletseM...","{'ref': 'master', 'ref_type': 'branch', 'maste...",True,2019-11-04T23:06:42Z,NaN


When we look at the data frame, we can see that there are dictionaries nested in several fields. We need to extract the information that is in these fields and add them to the data frame as columns. To do this, we are going to create our own flatten function that accepts a data frame and a list of columns that contain nested dictionaries in them. Our function is going to iterate through the columns and, for each column, it is going to:

    Turn the nested dictionaries into a data frame with a column for each key
    Assign column names to each column in this new data frame
    Add these new columns to the original data frame
    Drop the column with the nested dictionaries


In [4]:
def flatten(data, col_list):
    for column in col_list:
        flattened = pd.DataFrame(dict(data[column])).transpose()
        columns = [str(col) for col in flattened.columns]
        flattened.columns = [column + '_' + colname for colname in columns]
        data = pd.concat([data, flattened], axis=1)
        data = data.drop(column, axis=1)
    return data

Now that we have our function, let's apply it to the columns that have nested dictionaries and get back a revised data frame.

In [5]:
nested_columns = ['actor', 'org', 'payload', 'repo']

flat = flatten(data, nested_columns)
flat.head()

,id,type,public,created_at,actor_id,actor_login,actor_display_login,actor_gravatar_id,actor_url,actor_avatar_url,...,payload_master_branch,payload_description,payload_pusher_type,payload_forkee,payload_issue,payload_comment,payload_release,repo_id,repo_name,repo_url
0,10785261173,PushEvent,True,2019-11-04T23:06:42Z,18611758,victorjourne,victorjourne,,https://api.github.com/users/victorjourne,https://avatars.githubusercontent.com/u/18611758?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,218634132,victorjourne/test,https://api.github.com/repos/victorjourne/test
1,10785261172,PushEvent,True,2019-11-04T23:06:42Z,29196449,nickdurbin,nickdurbin,,https://api.github.com/users/nickdurbin,https://avatars.githubusercontent.com/u/29196449?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,218709067,nickdurbin/auth-friends,https://api.github.com/repos/nickdurbin/auth-f...
2,10785261161,PushEvent,True,2019-11-04T23:06:42Z,19600854,hwx438865,hwx438865,,https://api.github.com/users/hwx438865,https://avatars.githubusercontent.com/u/19600854?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202865472,hwx438865/carenginehub03,https://api.github.com/repos/hwx438865/carengi...
3,10785261145,PullRequestEvent,True,2019-11-04T23:06:42Z,41453073,concow,concow,,https://api.github.com/users/concow,https://avatars.githubusercontent.com/u/41453073?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,181036257,learn-co-students/css-graffiti-override-lab-on...,https://api.github.com/repos/learn-co-students...
4,10785261158,CreateEvent,True,2019-11-04T23:06:42Z,57375669,Aletse-Meza,Aletse-Meza,,https://api.github.com/users/Aletse-Meza,https://avatars.githubusercontent.com/u/57375669?,...,master,Biotechnology student,user,NaN,NaN,NaN,NaN,219613610,Aletse-Meza/AletseMeza,https://api.github.com/repos/Aletse-Meza/Alets...


Alternatively, we can flatten nested data using the function json_normalize. This function is part of the Pandas library. The function will flatten and rename each flattened column to the name of the original column and the name of the nested column separated by a period. For example actor.avatar_url.

Here is an example of how to use this function. Note that you have to import it separately in order to avoid using the full path when calling the function.

In [6]:
from pandas.io.json import json_normalize

results = response.json()
flattened_data = json_normalize(results)

flattened_data.head()

,id,type,public,created_at,actor.id,actor.login,actor.display_login,actor.gravatar_id,actor.url,actor.avatar_url,...,payload.release.author.site_admin,payload.release.prerelease,payload.release.created_at,payload.release.published_at,payload.release.assets,payload.release.tarball_url,payload.release.zipball_url,payload.release.body,payload.pull_request.head.repo.license,payload.pull_request.base.repo.license
0,10785261173,PushEvent,True,2019-11-04T23:06:42Z,18611758,victorjourne,victorjourne,,https://api.github.com/users/victorjourne,https://avatars.githubusercontent.com/u/18611758?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10785261172,PushEvent,True,2019-11-04T23:06:42Z,29196449,nickdurbin,nickdurbin,,https://api.github.com/users/nickdurbin,https://avatars.githubusercontent.com/u/29196449?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10785261161,PushEvent,True,2019-11-04T23:06:42Z,19600854,hwx438865,hwx438865,,https://api.github.com/users/hwx438865,https://avatars.githubusercontent.com/u/19600854?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10785261145,PullRequestEvent,True,2019-11-04T23:06:42Z,41453073,concow,concow,,https://api.github.com/users/concow,https://avatars.githubusercontent.com/u/41453073?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10785261158,CreateEvent,True,2019-11-04T23:06:42Z,57375669,Aletse-Meza,Aletse-Meza,,https://api.github.com/users/Aletse-Meza,https://avatars.githubusercontent.com/u/57375669?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Looks much cleaner, and now we have access to the information that was enclosed within those dictionaries. Sometimes multiple rounds of flattening will be required if the JSON data returned from the API you are working with has hierarchically nested data.
